<a href="https://colab.research.google.com/github/juliosdutra/Controle_de_Processos_UFES/blob/main/3.Transfer-functions/simulacao_sistema_segunda_ordem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<left><img src="https://github.com/juliosdutra/Controle_de_Processos_UFES/blob/main/imagens/cabe%C3%A7alho-geral.jpg?raw=true" width="700" height="175">
</left>

# Simulação de Sistema de Segunda Ordem

Nesta aplicação, simula-se a resposta ao degrau de um **sistema de segunda ordem** para observar como os parâmetros do sistema afetam seu comportamento. Sistemas de segunda ordem são comuns em engenharia, especialmente em controle de processos e sistemas dinâmicos. Esses sistemas são descritos por três parâmetros principais:

1. **Ganho (K)**
2. **Período (τ)**
3. **Fator de Amortecimento (ξ)**

Durante a simulação, é possível ajustar os valores de **ganho (K)**, **período (τ)** e **fator de amortecimento (ξ)**. Ao mudar esses valores, observe como a curva de resposta muda. Dessa forma, é possível entender como esses parâmetros impactam a dinâmica de sistemas de segunda ordem. Isso é fundamental para o projeto e controle de sistemas dinâmicos, permitindo otimizar a resposta conforme as necessidades de diferentes processos.

In [1]:
# @title
# Instalando os pacotes necessários
! pip install -qq control

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.3/578.3 kB 9.2 MB/s eta 0:00:00


In [2]:
# @title
# Simulação Interativa de Sistema de Segunda Ordem
from ipywidgets import interact, widgets, Button, Layout, VBox
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import control as ct
import numpy as np

# Função para plotar a resposta ao degrau do sistema de segunda ordem
def simula_2ordem(ganho, tau, fator_xi):
    K = ganho
    xi = fator_xi
    num = [K]
    den = [tau**2, 2*xi*tau, 1]
    G = ct.tf(num, den)
    t, y = ct.step_response(G)
    plt.figure(figsize=(8, 5))
    label_text = rf"$G(s) = \frac{{{num[0]}}}{{{round(den[0], 2)}s^2 + {round(den[1], 2)}s + {den[2]}}}$"
    plt.plot(t, y, color='dodgerblue', linewidth=2, label=label_text)
    plt.grid(True, which='both', linestyle='--', color='gray')
    plt.xlabel('Tempo, t', fontsize=12, color='black')
    plt.ylabel('Saída do sistema, y', fontsize=12, color='black')
    plt.title(f'Resposta ao Degrau', fontsize=14, color='black')
    plt.legend(loc='best', fontsize=14, facecolor='white', edgecolor='black')
    plt.gca().set_facecolor('#f0f0f0')  # Fundo claro para o gráfico
    plt.gca().spines['bottom'].set_color('black')
    plt.gca().spines['left'].set_color('black')
    plt.tick_params(axis='x', colors='black')
    plt.tick_params(axis='y', colors='black')
    plt.show()

# Sliders com cores e descrições
K_slider = widgets.FloatSlider(min=-5, max=5, step=0.01, value=1.0,
                                 description='Ganho', style={'description_width': 'initial'},
                                 layout=Layout(width='400px'), continuous_update=False)

tau_slider = widgets.FloatSlider(min=0.01, max=5, step=0.01, value=1.0,
                                 description='Período', style={'description_width': 'initial'},
                                 layout=Layout(width='400px'), continuous_update=False)

xi_slider = widgets.FloatSlider(min=0, max=1.5, step=0.01, value=0.7,
                                description='Amortecimento', style={'description_width': 'initial'},
                                layout=Layout(width='400px'), continuous_update=False)

# Função para redefinir sliders
def reset_sliders(b):
    K_slider.value = 1.0
    tau_slider.value = 1.0
    xi_slider.value = 0.7

# Botão de reset com estilo
reset_button = Button(description="Redefinir Valores",
                      button_style='primary', layout=Layout(width='200px'))

reset_button.on_click(reset_sliders)

# Layout completo com sliders e botão de reset
ui = VBox([reset_button])

# Interatividade com os sliders já definidos
interact(simula_2ordem, ganho=K_slider, tau=tau_slider, fator_xi=xi_slider)

# Exibindo a interface com os sliders e o botão de reset
display(ui)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Ganho', layout=Layout(width…

A seguir, explicam-se os efeitos de cada um desses parâmetros no comportamento do sistema.

1. Ganho (K)
O **ganho** está relacionado à amplitude da resposta do sistema. Ele determina o quanto a saída do sistema é amplificada ou atenuada em relação à entrada (no caso, o degrau). Aumentar o ganho resulta em uma curva de saída mais alta, já que o sistema amplifica a entrada.

2. Período (τ)
O **período** está relacionado ao tempo que o sistema leva para atingir a resposta final. Ele determina a velocidade de resposta do sistema. Com isso, aumentar o **τ** resulta em uma resposta mais lenta. A curva de saída demora mais tempo para atingir o valor final.

3. Fator de Amortecimento (ξ)
O **fator de amortecimento** controla o comportamento oscilatório do sistema. Ele determina se o sistema vai oscilar antes de se estabilizar e com que rapidez essas oscilações diminuem. Dessa forma:
  - Quando **ξ** é baixo (próximo de zero), o sistema oscila bastante antes de se estabilizar.
  - Quando **ξ** é alto (próximo de 1), o sistema se estabiliza rapidamente, sem oscilações.
  - Se **ξ** for maior que 1, o sistema se torna super-amortecido, não oscila, mas também leva mais tempo para atingir o valor final.

In [3]:
from ipywidgets import interact, widgets, Button, Layout, VBox
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from IPython.display import display, HTML

# Função para plotar a resposta ao degrau do sistema de segunda ordem
def simula_2ordem(ganho, tau, fator_xi):
    K = ganho
    xi = fator_xi
    num = [K]  # Numerador da função de transferência
    den = [tau**2, 2*xi*tau, 1]  # Denominador da função de transferência

    # Criando a função de transferência usando scipy.signal.TransferFunction
    system = signal.TransferFunction(num, den)

    # Obtendo a resposta ao degrau
    t, y = signal.step(system)

    # Plotando a resposta
    plt.figure(figsize=(8, 5))
    label_text = rf"$G(s) = \frac{{{num[0]}}}{{{round(den[0], 2)}s^2 + {round(den[1], 2)}s + {den[2]}}}$"
    plt.plot(t, y, color='dodgerblue', linewidth=2, label=label_text)
    plt.grid(True, which='both', linestyle='--', color='gray')
    plt.xlabel('Tempo, t', fontsize=12, color='black')
    plt.ylabel('Saída do sistema, y', fontsize=12, color='black')
    plt.title(f'Resposta ao Degrau', fontsize=14, color='black')
    plt.legend(loc='best', fontsize=14, facecolor='white', edgecolor='black')
    plt.gca().set_facecolor('#f0f0f0')  # Fundo claro para o gráfico
    plt.gca().spines['bottom'].set_color('black')
    plt.gca().spines['left'].set_color('black')
    plt.tick_params(axis='x', colors='black')
    plt.tick_params(axis='y', colors='black')
    plt.show()

# Sliders com cores e descrições
K_slider = widgets.FloatSlider(min=-5, max=5, step=0.01, value=1.0,
                                 description='Ganho', style={'description_width': 'initial'},
                                 layout=Layout(width='400px'), continuous_update=False)

tau_slider = widgets.FloatSlider(min=0.01, max=5, step=0.01, value=1.0,
                                 description='Período', style={'description_width': 'initial'},
                                 layout=Layout(width='400px'), continuous_update=False)

xi_slider = widgets.FloatSlider(min=0, max=1.5, step=0.01, value=0.7,
                                description='Amortecimento', style={'description_width': 'initial'},
                                layout=Layout(width='400px'), continuous_update=False)

# Função para redefinir sliders
def reset_sliders(b):
    K_slider.value = 1.0
    tau_slider.value = 1.0
    xi_slider.value = 0.7

# Botão de reset com estilo
reset_button = Button(description="Redefinir Valores",
                      button_style='primary', layout=Layout(width='200px'))

reset_button.on_click(reset_sliders)

# Layout completo com sliders e botão de reset
ui = VBox([reset_button])

# Interatividade com os sliders já definidos
# Não estamos passando os sliders para o interact, assim ele não cria os sliders automaticamente
interact(simula_2ordem, ganho=K_slider, tau=tau_slider, fator_xi=xi_slider)

# Exibindo a interface com os sliders e o botão de reset
display(ui)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='Ganho', layout=Layout(width…